In [1]:
# STANDARD LIBRARIES
import os
import warnings
warnings.filterwarnings("ignore")

# THIRD PARTY LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pydataset
import scipy.stats as stats

#statistical tests
from scipy import stats
from scipy.stats import pearsonr, spearmanr



# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.feature_selection import RFE
import matplotlib
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from math import sqrt

In [2]:
import acquire 
import prepare

In [3]:
df = acquire.get_fifa_data()

In [4]:
df = prepare.prepped_data(df)

Before dropping nulls, 20700 rows, 111 cols
After dropping nulls. 19457 rows. 66 cols
After dropping leagues. 15373 rows. 79 cols


In [8]:
train, validate, test = prepare.split(df)

Train: 10453 rows, 79 cols
Validate: 2614 rows, 79 cols
Test: 2306 rows, 79 cols


In [6]:
df.club_position.value_counts()
#goal keeper 
#offense 
#defense 

SUB    6428
RES    2411
GK      633
RCB     589
LCB     576
RB      458
LB      445
ST      419
LCM     400
RCM     400
LM      358
RM      356
CAM     269
LDM     194
RDM     193
RS      192
LS      186
LW      171
RW      164
CB      148
CDM     148
RWB      81
LWB      77
LF       26
LAM      22
RAM      20
CF        9
Name: club_position, dtype: int64

In [35]:
#GK
goalkeepers = train[(train.club_position == 'GK')]

In [42]:
#ST, RS, LS, LW, RW, LF, CF
forwards = train[(train.club_position == 'ST') | (train.club_position == 'RS') | (train.club_position == 'LS') | (train.club_position == 'LW') | (train.club_position == 'RW') | (train.club_position == 'LF') | (train.club_position == 'CF')]


In [43]:
#LCM, RCM, RM, LM, CAM, LDM, RDM, CDM, LAM, RAM 
midfielders = train[(train.club_position == 'LCM') | (train.club_position == 'RCM') | (train.club_position == 'RM') | (train.club_position == 'LM') | (train.club_position == 'CAM') | (train.club_position == 'LDM') | (train.club_position == 'RDM') | (train.club_position == 'CDM') | (train.club_position == 'LAM') | (train.club_position == 'RAM')] 


In [44]:
#RCB, LCB, LB, RB, CB, RWB, LWB
defenders = train[(train.club_position == 'RCB') | (train.club_position == 'LCB') | (train.club_position == 'LB') | (train.club_position == 'RB') | (train.club_position == 'CB') | (train.club_position == 'RWB') | (train.club_position == 'LWB')]

In [45]:
train_reserve = train[train.club_position == 'RES']
train_substitute = train[train.club_position == 'SUB']

In [46]:
train_reserve.shape

(1640, 79)

In [47]:
train_substitute.shape

(4361, 79)

In [48]:
pd.set_option("display.max_rows", None)
train_reserve.player_positions.value_counts()

GK              254
CB              202
ST              132
CM               94
LB               71
CDM, CM          60
RB               54
CAM              42
CM, CDM          40
CDM              39
LM               36
RM               33
CAM, CM          26
CM, CAM          19
RW               16
CB, RB           15
CB, CDM          15
CB, LB           14
LB, CB           13
RM, LM           13
RB, CB           12
ST, CAM          11
LB, LM           11
LM, RM           11
CDM, CB          11
LW               10
RB, RM           10
RB, LB            9
RW, LW            9
LW, ST            8
RM, ST            8
ST, RW            8
ST, CF            7
ST, LM            7
ST, RM            6
CAM, ST           6
LM, LW            6
RW, ST            5
LB, LWB           5
RM, CAM           5
CM, CDM, CAM      5
LB, RB            5
ST, LW            5
LM, CAM           5
CAM, RM           5
LM, RM, CAM       5
LM, CF            4
CAM, LW           4
LM, ST            4
CAM, CM, LM       4


In [41]:
train_reserve_goalkeepers = train_reserve[train_reserve.player_positions == 'GK']

In [49]:
#ST, RS, LS, LW, RW, LF, CF
train_reserve_forwards = train_reserve[(train_reserve.player_positions == 'ST') | (train_reserve.player_positions == 'RW') | (train_reserve.player_positions == 'ST, CAM') | (train_reserve.player_positions == 'LW') | (train_reserve.player_positions == 'RW, LW') | (train_reserve.player_positions == 'LW, ST') | (train_reserve.player_positions == 'ST, RW') | (train_reserve.player_positions == 'ST, CF') | (train_reserve.player_positions == 'ST, LM') | (train_reserve.player_positions == 'ST, RM') | (train_reserve.player_positions == 'RW, ST') | (train_reserve.player_positions == 'ST, LW') | (train_reserve.player_positions == 'RW, RM') | (train_reserve.player_positions == 'LW, CAM') | (train_reserve.player_positions == 'LW, RW') | (train_reserve.player_positions == 'RW, CM') | (train_reserve.player_positions == 'RW, LW, CAM') | (train_reserve.player_positions == 'ST, RW, LW') | (train_reserve.player_positions == 'LW, LM') | (train_reserve.player_positions == 'CF') | (train_reserve.player_positions == 'RW, CAM') | (train_reserve.player_positions == 'ST, LW, RW') | (train_reserve.player_positions == 'CF, ST, CAM') | (train_reserve.player_positions == 'LW, LB') | (train_reserve.player_positions == 'CF, RM, ST') | (train_reserve.player_positions == 'LW, LM, LB') | (train_reserve.player_positions == 'CF, ST') | (train_reserve.player_positions == 'RW, CAM, ST') | (train_reserve.player_positions == 'ST, RM, LM') | (train_reserve.player_positions == 'LW, RW, CAM') | (train_reserve.player_positions == 'ST, LW, LM') | (train_reserve.player_positions == 'LW, CF, CAM') | (train_reserve.player_positions == 'CF, CAM, ST') | (train_reserve.player_positions == 'LW, CF, RW') | (train_reserve.player_positions == 'LW, RW, ST') | (train_reserve.player_positions == 'RW, LW, ST') | (train_reserve.player_positions == 'CF, CAM, LM') | (train_reserve.player_positions == 'ST, LM, LW') | (train_reserve.player_positions == 'RW, LW, CF') | (train_reserve.player_positions == 'LW, CAM, RW') | (train_reserve.player_positions == 'RW, ST, LM') | (train_reserve.player_positions == 'RW, ST, LW') | (train_reserve.player_positions == 'CF, LM') | (train_reserve.player_positions == 'ST, LM, RM') | (train_reserve.player_positions == 'RW, RM, RB') | (train_reserve.player_positions == 'ST, CAM, RM') | (train_reserve.player_positions == 'RW, LW, LM') | (train_reserve.player_positions == 'RW, RM, LM') | (train_reserve.player_positions == 'RW, LW, RM')        ]

In [50]:
#LCM, RCM, RM, LM, CAM, LDM, RDM, CDM, LAM, RAM 
train_reserve_midfielders = train_reserve[(train_reserve.player_positions == 'CM') | (train_reserve.player_positions == 'CDM, CM') | (train_reserve.player_positions == 'CAM') | (train_reserve.player_positions == 'CM, CDM') | (train_reserve.player_positions == 'CDM') | (train_reserve.player_positions == 'LM') | (train_reserve.player_positions == 'RM') | (train_reserve.player_positions == 'CAM, CM') | (train_reserve.player_positions == 'CM, CAM') | (train_reserve.player_positions == 'RM, LM') | (train_reserve.player_positions == 'LM, RM') | (train_reserve.player_positions == 'CDM, CB') | (train_reserve.player_positions == 'RM, ST') | (train_reserve.player_positions == 'CAM, ST') | (train_reserve.player_positions == 'LM, LW') | (train_reserve.player_positions == 'RM, CAM') | (train_reserve.player_positions == 'CM, CDM, CAM') | (train_reserve.player_positions == 'LM, CAM') | (train_reserve.player_positions == 'CAM, RM') | (train_reserve.player_positions == 'LM, RM, CAM') | (train_reserve.player_positions == 'LM, CF') | (train_reserve.player_positions == 'CAM, LW') | (train_reserve.player_positions == 'LM, ST') | (train_reserve.player_positions == 'CAM, CM, LM') | (train_reserve.player_positions == 'RM, CM') | (train_reserve.player_positions == 'RM, LM, CM') | (train_reserve.player_positions == 'RM, RWB') | (train_reserve.player_positions == 'CAM, RM, LM') | (train_reserve.player_positions == 'CAM, CDM') | (train_reserve.player_positions == 'CAM, LM') | (train_reserve.player_positions == 'RM, LM, CAM') | (train_reserve.player_positions == 'CM, CAM, RM') | (train_reserve.player_positions == 'CDM, CM, CB') | (train_reserve.player_positions == 'CAM, CF') | (train_reserve.player_positions == 'CM, CAM, CDM') | (train_reserve.player_positions == 'CM, RM') | (train_reserve.player_positions == 'CAM, RW, LW') | (train_reserve.player_positions == 'LM, CAM, RM') | (train_reserve.player_positions == 'CDM, RB') | (train_reserve.player_positions == 'RM, LM, ST') | (train_reserve.player_positions == 'LM, RM, CF') | (train_reserve.player_positions == 'CAM, LM, RM') | (train_reserve.player_positions == 'CM, CB') | (train_reserve.player_positions == 'RM, ST, RB') | (train_reserve.player_positions == 'CDM, CB, CM') | (train_reserve.player_positions == 'LM, ST, RM') | (train_reserve.player_positions == 'RM, CF') | (train_reserve.player_positions == 'LM, CAM, CM') | (train_reserve.player_positions == 'CAM, CF, RW') | (train_reserve.player_positions == 'LM, LW, RW') | (train_reserve.player_positions == 'CM, RW') | (train_reserve.player_positions == 'RM, RW') | (train_reserve.player_positions == 'CM, RM, LM') | (train_reserve.player_positions == 'RM, ST, LB') | (train_reserve.player_positions == 'CM, CAM, RW') | (train_reserve.player_positions == 'CAM, CM, RM') | (train_reserve.player_positions == 'CAM, CM, CF') | (train_reserve.player_positions == 'CDM, LB, LM') | (train_reserve.player_positions == 'LM, ST, LB') | (train_reserve.player_positions == 'RM, LWB') | (train_reserve.player_positions == 'LM, CDM') | (train_reserve.player_positions == 'CM, LM, RM') | (train_reserve.player_positions == 'CAM, CM, ST') | (train_reserve.player_positions == 'RM, CM, LM') | (train_reserve.player_positions == 'CAM, RM, RW') | (train_reserve.player_positions == 'CDM, CM, CAM') | (train_reserve.player_positions == 'CDM, CM, RM') | (train_reserve.player_positions == 'CAM, ST, CM') | (train_reserve.player_positions == 'RM, RB') | (train_reserve.player_positions == 'CM, RM, CAM') | (train_reserve.player_positions == 'RM, CAM, CM') | (train_reserve.player_positions == 'CAM, RW') | (train_reserve.player_positions == 'CM, CF, LM') | (train_reserve.player_positions == 'RM, RB, RWB') | (train_reserve.player_positions == 'CDM, LB, CB') | (train_reserve.player_positions == 'LM, LWB, LB') | (train_reserve.player_positions == 'CAM, RM, CM') | (train_reserve.player_positions == 'CDM, RB, RWB') | (train_reserve.player_positions == 'CM, RB') | (train_reserve.player_positions == 'CM, CDM, RB') | (train_reserve.player_positions == 'RM, LM, RW') | (train_reserve.player_positions == 'RM, LW') | (train_reserve.player_positions == 'CDM, CAM, CM') | (train_reserve.player_positions == 'LM, RM, LWB') | (train_reserve.player_positions == 'RM, RWB, RB') | (train_reserve.player_positions == 'CAM, ST, RW') | (train_reserve.player_positions == 'LM, CM') | (train_reserve.player_positions == 'LM, ST, CAM') | (train_reserve.player_positions == 'CM, CDM, LWB') | (train_reserve.player_positions == 'LM, CAM, ST') | (train_reserve.player_positions == 'CDM, CB, RM') | (train_reserve.player_positions == 'CDM, CM, RB') | (train_reserve.player_positions == 'CM, LB') | (train_reserve.player_positions == 'CAM, LW, LM') | (train_reserve.player_positions == 'CAM, CM, LW') | (train_reserve.player_positions == 'CDM, RB, LB') | (train_reserve.player_positions == 'LM, RM, LB') | (train_reserve.player_positions == 'LM, LB, RM') | (train_reserve.player_positions == 'RM, ST, CAM') | (train_reserve.player_positions == 'CAM, LM, LW') | (train_reserve.player_positions == 'CM, LW') | (train_reserve.player_positions == 'RM, RB, RW') | (train_reserve.player_positions == 'CM, CDM, RM') | (train_reserve.player_positions == 'CM, LM') | (train_reserve.player_positions == 'LM, LB')      ]

In [51]:
#RCB, LCB, LB, RB, CB, RWB, LWB
train_reserve_defenders =  train_reserve[(train_reserve.player_positions == 'CB') | (train_reserve.player_positions == 'LB') | (train_reserve.player_positions == 'RB') | (train_reserve.player_positions == 'CB, RB') | (train_reserve.player_positions == 'CB, CDM') | (train_reserve.player_positions == 'CB, LB') | (train_reserve.player_positions == 'LB, CB') | (train_reserve.player_positions == 'RB, CB') | (train_reserve.player_positions == 'LB, LM') | (train_reserve.player_positions == 'RB, RM') | (train_reserve.player_positions == 'RB, LB') | (train_reserve.player_positions == 'LB, LWB') | (train_reserve.player_positions == 'LB, RB') | (train_reserve.player_positions == 'LWB') | (train_reserve.player_positions == 'RB, RWB') | (train_reserve.player_positions == 'RB, CM') | (train_reserve.player_positions == 'CB, CDM, CM') | (train_reserve.player_positions == 'CB, CM, CDM') | (train_reserve.player_positions == 'RWB, RB') | (train_reserve.player_positions == 'LWB, LB') | (train_reserve.player_positions == 'LB, LM, CDM') | (train_reserve.player_positions == 'RB, CDM, CM') | (train_reserve.player_positions == 'LB, LWB, LM') | (train_reserve.player_positions == 'CB, RB, LB') | (train_reserve.player_positions == 'LB, LW') | (train_reserve.player_positions == 'CB, ST') | (train_reserve.player_positions == 'RWB, RB, RM') | (train_reserve.player_positions == 'LB, CB, LM') | (train_reserve.player_positions == 'RB, CAM, RM') | (train_reserve.player_positions == 'LB, CDM') | (train_reserve.player_positions == 'CB, LB, RB') | (train_reserve.player_positions == 'LB, RB, CB') | (train_reserve.player_positions == 'RB, LB, RWB') | (train_reserve.player_positions == 'LWB, LB, LM') | (train_reserve.player_positions == 'CB, CM') | (train_reserve.player_positions == 'LB, CB, CDM') | (train_reserve.player_positions == 'LB, RB, CM') | (train_reserve.player_positions == 'LB, RB, RWB') | (train_reserve.player_positions == 'RB, RW') | (train_reserve.player_positions == 'RB, CB, CDM') | (train_reserve.player_positions == 'RB, RM, RW') | (train_reserve.player_positions == 'RB, RWB, CB') | (train_reserve.player_positions == 'RWB') | (train_reserve.player_positions == 'RB, RWB, RW') | (train_reserve.player_positions == 'RB, CDM') | (train_reserve.player_positions == 'LB, LM, RM') | (train_reserve.player_positions == 'CB, RB, CDM') | (train_reserve.player_positions == 'LWB, LM, LB') | (train_reserve.player_positions == 'RB, CM, RWB') | (train_reserve.player_positions == 'RB, RWB, RM') | (train_reserve.player_positions == 'LB, LM, RB') | (train_reserve.player_positions == 'CB, CDM, RB') | (train_reserve.player_positions == 'CB, LWB') | (train_reserve.player_positions == 'LB, LM, LWB')  ] 

In [52]:
train_reserve_goalkeepers.shape

(254, 79)

In [53]:
train_reserve_forwards.shape

(282, 79)

In [54]:
train_reserve_midfielders.shape

(608, 79)

In [55]:
train_reserve_defenders.shape

(496, 79)

In [ ]:
#ends in b backline 
#cdm backline 
d in middle clue backline

In [ ]:
Forward
Striker, Center Forward, Wingers

In [ ]:
Midfielder
Left Mid, Right Mid, Center Mid, Attacking Mid, Defensive Mid

In [ ]:
Defender
Center back, Full Back (Left Back, Right Back), Wing Backs, Sweeper (RARE)s

In [57]:
train_substitute.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,weight_bins,year_joined,seniority,league_encoded,club_position_encoded,work_rate_encoded,preferred_foot_encoded,age_bins_encoded,weight_bins_encoded,body_type_encoded
5974,230506,Adão Salvadaldo,"ST, LW",69,72,"1,900,000.00","8,000.00",25,174,69,...,average,2021,1,25,12,5,1,2,2,3
6119,238223,B. Bvuma,GK,69,73,"1,500,000.00",500.00,26,186,73,...,average,2017,5,33,12,5,2,2,2,5
19170,264075,K. Kundaikar,CB,49,58,"100,000.00",500.00,22,182,76,...,average,2021,1,40,12,8,2,2,2,2
20651,226537,V. Janssen,ST,76,78,"9,500,000.00","42,000.00",26,184,79,...,average,2019,2,10,12,8,1,2,2,2
9741,260024,V. Ayala,"LB, LM",66,66,"600,000.00",500.00,32,179,80,...,average,2020,2,34,12,4,1,1,2,2


In [58]:
train_substitute.player_positions

5974           ST, LW
6119               GK
19170              CB
20651              ST
9741           LB, LM
8873               CM
12661         CDM, CM
12511         CAM, LM
3713          CM, CDM
17658          ST, LW
10055         CM, CDM
19393              LB
12569          LM, RM
10743         CM, CDM
1580           CB, LB
13829          LB, LM
13962          RW, LW
6812          CM, CDM
6692               CB
2545               GK
5177          CAM, CM
15500              ST
18625      LM, RM, ST
9587               CB
13020         CAM, CM
8582       LM, LW, ST
13902              ST
6613          CDM, CM
9680               GK
14342              CB
20491              GK
16847              ST
15826         CAM, CM
11517    CM, CDM, CAM
13585              CB
1754               CB
8417      LW, LM, CAM
7624           CM, RW
10816              ST
10655          LM, ST
13042          CM, RM
15402              LM
8638               ST
15601     RWB, RB, CB
11275     CM, CDM, CB
18636     

In [65]:
train_substitute['player_positions'] = train_substitute['player_positions'].str.split(',').str[0]


In [66]:
train_substitute.head(100)

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,weight_bins,year_joined,seniority,league_encoded,club_position_encoded,work_rate_encoded,preferred_foot_encoded,age_bins_encoded,weight_bins_encoded,body_type_encoded
5974,230506,Adão Salvadaldo,ST,69,72,"1,900,000.00","8,000.00",25,174,69,...,average,2021,1,25,12,5,1,2,2,3
6119,238223,B. Bvuma,GK,69,73,"1,500,000.00",500.00,26,186,73,...,average,2017,5,33,12,5,2,2,2,5
19170,264075,K. Kundaikar,CB,49,58,"100,000.00",500.00,22,182,76,...,average,2021,1,40,12,8,2,2,2,2
20651,226537,V. Janssen,ST,76,78,"9,500,000.00","42,000.00",26,184,79,...,average,2019,2,10,12,8,1,2,2,2
9741,260024,V. Ayala,LB,66,66,"600,000.00",500.00,32,179,80,...,average,2020,2,34,12,4,1,1,2,2
8873,204220,C. Edwards,CM,66,67,"975,000.00","3,000.00",27,180,70,...,average,2019,3,16,12,5,2,2,2,3
12661,234982,M. Christensen,CDM,63,68,"675,000.00","3,000.00",23,190,74,...,average,2016,6,20,12,3,2,2,2,5
12511,223305,P. Mystkowski,CAM,63,66,"650,000.00","2,000.00",23,174,62,...,slim,2014,8,14,12,4,1,2,1,3
3713,239845,K. Galanopoulos,CM,72,78,"3,700,000.00",850.00,23,172,72,...,average,2015,7,31,12,8,2,2,2,2
17658,260484,S. Ramírez,ST,56,70,"350,000.00",500.00,19,175,66,...,slim,2017,5,38,12,5,1,2,1,2


In [67]:
train_substitute.player_positions.value_counts()

CB     655
ST     648
CM     501
GK     490
CDM    329
LB     316
RB     298
CAM    273
LM     259
RM     248
RW     148
LW     106
RWB     34
CF      34
LWB     22
Name: player_positions, dtype: int64

In [69]:
train_substitute_goalkeeper = train_substitute[train_substitute.player_positions == 'GK']

In [71]:
train_substitute_forward = train_substitute[(train_substitute.player_positions == 'ST') | (train_substitute.player_positions == 'RW') | (train_substitute.player_positions == 'LW') | (train_substitute.player_positions == 'CF')]

In [74]:
train_substitute_midfielders =  train_substitute[(train_substitute.player_positions == 'CM') | (train_substitute.player_positions == 'CDM') | (train_substitute.player_positions == 'CAM') | (train_substitute.player_positions == 'LM') | (train_substitute.player_positions == 'RM')]

In [75]:
train_substitute_defenders =  train_substitute[(train_substitute.player_positions == 'CB') | (train_substitute.player_positions == 'LB') | (train_substitute.player_positions == 'RB') | (train_substitute.player_positions == 'RWB') | (train_substitute.player_positions == 'LWB')]

In [ ]:
#vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)


In [80]:
goalkeeper_df = pd.concat([goalkeepers, train_substitute_goalkeeper, train_reserve_goalkeepers], axis=0)

In [81]:
forward_df = pd.concat([forwards, train_substitute_forward, train_reserve_forwards], axis=0)

In [82]:
midfielder_df = pd.concat([midfielders, train_substitute_midfielders, train_reserve_midfielders], axis=0)

In [83]:
defender_df = pd.concat([defenders, train_substitute_defenders, train_reserve_defenders], axis=0)

In [85]:
goalkeeper_df.shape

(1179, 79)

In [86]:
forward_df.shape

(2003, 79)

In [87]:
midfielder_df.shape

(3814, 79)

In [88]:
defender_df.shape

(3457, 79)